In [1]:
import cebra
import torch
import torch.utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from utils import TensorDataset, SimpleTensorDataset, SupervisedNNSolver
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

# Data Load

In [2]:
df = pd.read_hdf("data/ID18150/Day2/DataFrame_Imaging_dFF_18150_day2.h5")
# Get a list of columns whose names are of numerical type
numerical_columns = [col for col in df.columns if type(col) == int]

#Feature matrix
X = df[numerical_columns].values
t = df.Time.to_list()

In [3]:
delta = 5000
def get_x_ticks(L:int):
    x_ticks = np.arange(0,L,delta)
    x_tick_labels = [f"{t[i]/100:.2f}" for i in x_ticks]
    return x_ticks, x_tick_labels

In [4]:
split = int(0.2*len(X))
X_train, X_test = X[:-split], X[-split:]
y_train, y_test = df["Pump"].values[:-split], df["Pump"].values[-split:]
X_train.shape, X_test.shape

((52796, 709), (13198, 709))

In [5]:
# x_ticks, x_tick_labels = get_x_ticks(X.shape[0])
# fig, ax = plt.subplots(1, 1, figsize=(15, 6))

# ax = sns.heatmap(X.T, ax=ax, cmap="gray_r")
# ax.set_xticks(x_ticks)
# ax.set_xticklabels(x_tick_labels)

# v_bar = X_train.shape[0]
# ax.axvline(v_bar, color="red")

# ax.set_title("Train and Test split")

# plt.show()

# CEBRA Encoder

## Dataset Preparation

In [6]:
with device:
    X_train_tensor = torch.from_numpy(X_train).type(torch.FloatTensor)
    X_test_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)
    y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)
    y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)

# Models

In [7]:
class ModelPipeline:
    def __init__(self, model_name, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, num_units, latent_dimension = 8):
        self.model_name = model_name
        self.X_train_tensor = X_train_tensor
        self.y_train_tensor = y_train_tensor
        self.X_test_tensor = X_test_tensor
        self.y_test_tensor = y_test_tensor
        self.latent_dimension = latent_dimension
        self.num_units = num_units
        self.train_dataset = TensorDataset(
            neural = X_train_tensor,
            discrete = y_train_tensor,
        )
        self.test_dataset = TensorDataset(
            neural = X_test_tensor,
            discrete = y_test_tensor
        )
        with device:
            self.model = cebra.models.init(
                name = model_name,
                num_neurons = self.train_dataset.neural.shape[1],
                num_units = num_units,
                num_output = latent_dimension
            )
            self.train_dataset.configure_for(self.model)
            self.test_dataset.configure_for(self.model)
    def train_embedding(self, learning_rate, batch_size, steps = 1000, verbose = True):
        self.batch_size = batch_size
        self.steps = steps
        self.learning_rate = learning_rate
        self.steps = steps
        with device:
            criterion = cebra.models.criterions.LearnableCosineInfoNCE()
            optimizer = torch.optim.Adam(
                list(self.model.parameters()) + list(criterion.parameters()),
                lr = learning_rate
            )
            self.embedding_solver = cebra.solver.SingleSessionSolver(
                model = self.model,
                criterion = criterion,
                optimizer = optimizer,
                tqdm_on = verbose
            )
            train_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.train_dataset,
                num_steps = steps,
                batch_size = batch_size,
                prior = "empirical"
            )
            self.embedding_solver.fit(loader=train_loader)
    def train_decoder(self, verbose = True):
        with device:
            self.simple_train_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_train_tensor.type(torch.FloatTensor),
                    labels = self.y_train_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                ),
                batch_size = self.batch_size,
                shuffle = True
            )
            self.binaryClassifier = torch.nn.Sequential(
                self.model,
                torch.nn.Linear(self.latent_dimension,self.latent_dimension),
                torch.nn.GELU(),
                torch.nn.Linear(self.latent_dimension,1),
                torch.nn.GELU(),
                torch.nn.Linear(1,1) #Logit output
            ).to(device)
            self.decoder_solver = SupervisedNNSolver(
                model = self.binaryClassifier,
                criterion = torch.nn.BCEWithLogitsLoss(),
                optimizer = torch.optim.Adam(self.binaryClassifier.parameters(), lr = self.learning_rate)
            )
            self.decoder_solver.fit(self.simple_train_dataloader, num_steps = self.steps)
    def train(self, learning_rate, batch_size, steps, verbose = True):
        self.train_embedding(learning_rate, batch_size, steps, verbose)
        self.train_decoder(verbose = verbose)
    def score(self, verbose = True):
        with device:
            test_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.test_dataset,
                num_steps = self.steps,
                batch_size = self.batch_size,
                prior = "empirical"
            )
            self.embedding_score = self.embedding_solver.validation(test_loader)
            self.simple_test_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_test_tensor.type(torch.FloatTensor),
                    labels = self.y_test_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                    ),
                batch_size = self.batch_size,
                shuffle = True
                )
            self.decoder_score = self.decoder_solver.validation(self.simple_test_dataloader)
            try:
                # Fit an exponential decay to the history and get the R2 value
                def exp_decay(x, a, b, c):
                    return a * np.exp(-b * x) + c

                # Fit the exponential decay to the solver history
                x_data = np.arange(len(self.embedding_solver.history))
                y_data = self.embedding_solver.history
                y0 = y_data[0]
                yf = y_data[-1]
                popt, _ = curve_fit(exp_decay, x_data, y_data, p0=(y0 - yf, 1e-6, yf))

                # Calculate the R2 value
                y_pred = exp_decay(x_data, *popt)
                r2 = r2_score(y_data, y_pred)
                if verbose:
                    print(f"Expontial decay R2: {r2}, embedding score: {self.embedding_score}, decoder score: {self.decoder_score}")
                return -r2 + self.embedding_score + 2*self.decoder_score
            except:
                return float("inf")
        

In [9]:
models_to_try = list(filter(lambda x: "offset" == x[:6] and not "mse" == x[-3:] and not "subsample" == x[-9:],cebra.models.get_options()))
models_to_try

['offset10-model',
 'offset5-model',
 'offset1-model',
 'offset1-model-v2',
 'offset1-model-v3',
 'offset1-model-v4',
 'offset1-model-v5',
 'offset36-model',
 'offset36-model-dropout',
 'offset36-model-more-dropout']

In [ ]:
def experiment(trial):
    model_name = trial.suggest_categorical("model_name", models_to_try)
    num_units = trial.suggest_int("num_units", 1, X_train.shape[1])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int("batch_size", 50, 512, log=True)
    model = ModelPipeline(
        model_name,
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test_tensor=y_test_tensor,
        num_units=num_units,
        latent_dimension=8
    )
    model.train(learning_rate, batch_size, 500)
    score = model.score()
    
    # Save the best model to disk
    if not hasattr(experiment, "best_score") or score < experiment.best_score:
        experiment.best_score = score
        experiment.pipeline = model
        torch.save(model.model.state_dict(), "./data/models/best_model.pth")
    
    return score

In [ ]:
study = optuna.create_study(storage="sqlite:///data/ID18150/Day2/optuna.db", study_name="cebra_offsets", direction="minimize", load_if_exists=True)
study.optimize(experiment, n_trials=100)

study.best_params

[I 2024-10-20 00:10:46,177] A new study created in RDB with name: cebra_offsets
pos: -1.1369 neg:  6.7022 total:  5.5653 temperature:  0.8271: 100%|██████████| 500/500 [00:26<00:00, 18.61it/s]
100%|██████████| 500/500 [00:06<00:00, 74.03it/s]
/home/icaro/Documents/doctorate/dimensionality-reduction/utils.py:78: UserWarning: You should pass at least one of the arguments 'continuous' or 'discrete'.
  warnings.warn(
[I 2024-10-20 00:11:21,804] Trial 0 finished with value: 5.271162625314428 and parameters: {'model_name': 'offset1-model-v3', 'num_units': 617, 'learning_rate': 0.0004034283040169691, 'batch_size': 279, 'n_neighbors': 14}. Best is trial 0 with value: 5.271162625314428.


Expontial decay R2: 0.47565444696226233, embedding score: 5.649832680702209, decoder score: 0.04849219578724051


pos: -0.9632 neg:  7.1052 total:  6.1420 temperature:  1.0379: 100%|██████████| 500/500 [16:46<00:00,  2.01s/it]
/home/icaro/Documents/doctorate/dimensionality-reduction/utils.py:78: UserWarning: You should pass at least one of the arguments 'continuous' or 'discrete'.
  warnings.warn(
